In [22]:
import geopandas as gpd
import rasterio
import cv2
import numpy as np
import os
import tensorflow

In [17]:
# Load shapefile
shapefile_path = 'C:\\Users\\Student\\Documents\\101_walrus_mres\\GIS\\torellneset_train_08.shp'
gdf = gpd.read_file(shapefile_path)

# Load satellite image
image_path = "C:\\Users\\Student\\Documents\\101_walrus_mres\\planet_skysat_pairs\\planet_skysat_pairs\\torellneset_psscene_analytic_8b_sr_udm2\\PSScene\\20230824_113933_07_2438_3B_AnalyticMS_SR_8b_harmonized_clip.tif"

with rasterio.open(image_path) as src:
    image = src.read()
    image = np.moveaxis(image, 0, -1)  # Move channels to the last dimension for OpenCV

In [6]:
# Generate masks from shapefile
def create_mask(image_shape, gdf, transform):
    mask = np.zeros((image_shape[0], image_shape[1]), dtype=np.uint8)
    for _, row in gdf.iterrows():
        geometry = row['geometry']
        coords = [(x, y) for x, y in geometry.exterior.coords]
        coords = [~transform * (x, y) for x, y in coords]
        coords = np.array(coords, dtype=np.int32)
        cv2.fillPoly(mask, [coords], 1)
    return mask

with rasterio.open(image_path) as src:
    transform = src.transform
    mask = create_mask((src.height, src.width), gdf, transform)


In [11]:
# Augment data for better model performance 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [26]:
import tifffile as tiff
import numpy as np
import os

def load_tiff_image(image_path):
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Error: The file at {image_path} does not exist.")
    
    img = tiff.imread(image_path)
    shape = img.shape
    
    # Handling multi-dimensional TIFF files
    if len(shape) == 2:  # Grayscale image
        height, width = shape
        channels = 1
    elif len(shape) == 3:  # Color image with channels
        height, width, channels = shape
    else:
        raise ValueError(f"Unexpected image shape: {shape}")

    return height, width, channels

# Example 
image_path = "C:\\Users\\Student\\Documents\\101_walrus_mres\\planet_skysat_pairs\\planet_skysat_pairs\\torellneset_psscene_analytic_8b_sr_udm2\\PSScene\\20230824_113933_07_2438_3B_AnalyticMS_SR_8b_harmonized_clip.tif"
try:
    height, width, channels = load_tiff_image(image_path)
    print(f"Image shape: {height}x{width}x{channels}")
except FileNotFoundError:
    print(f"Error: The file at {image_path} does not exist.")
except ValueError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"Error: {e}")


Image shape: 8x1327x1640


In [ ]:
# Define the model 
import tensorflow as tf
from tensorflow.keras import layers, models

def create_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

input_shape = (1640, 1327, 8)  # Adjust for image shape
model = create_model(input_shape)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


C:\Users\Student\Documents\walrus_code\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [3]:
# Training data 
from sklearn.model_selection import train_test_split

# if images and masks are lists
images = [image]  # List of images
masks = [mask]    # List of corresponding masks

train_images, val_images, train_masks, val_masks = train_test_split(images, masks, test_size=0.2, random_state=42)

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_masks))
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_masks))

# Batch and shuffle datasets
BATCH_SIZE = 16
train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(buffer_size=1000)
val_dataset = val_dataset.batch(BATCH_SIZE)


In [ ]:
#Train the model 
history = model.fit(train_dataset, epochs=20, validation_data=val_dataset)


In [ ]:
# Evailuate the mdoel 
test_loss, test_acc = model.evaluate(val_dataset)
print(f'Test accuracy: {test_acc}')

# Save the model
model.save('walrus_detection_model.h5')


In [ ]:
# Inference on new images 
# Load the model
model = tf.keras.models.load_model('walrus_detection_model.h5')

# Predict on a new image
new_image_path = ''
with rasterio.open(new_image_path) as src:
    new_image = src.read()
    new_image = np.moveaxis(new_image, 0, -1)  # Move channels to the last dimension

# Preprocess the image as needed (resize, normalize, google best one)
new_image = cv2.resize(new_image, (128, 128))
new_image = np.expand_dims(new_image, axis=0)  # Add batch dimension

# Perform prediction
prediction = model.predict(new_image)
